In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('hello spark').master('local').getOrCreate()

In [3]:

checkin_df = spark.read.json('D:/china/intershipBigData/yelp/yelp_academic_dataset_checkin.json')
business_df = spark.read.json('D:/china/intershipBigData/yelp/yelp_academic_dataset_business.json')
joined_df = checkin_df.join(business_df, checkin_df.business_id == business_df.business_id)
city_checkin_frequency = joined_df.groupBy("city").agg(count("*").alias("checkin_count")).orderBy(col("checkin_count").desc())
city_checkin_frequency.show(20)




+----------------+-------------+
|            city|checkin_count|
+----------------+-------------+
|    Philadelphia|        12892|
|           Tampa|         8005|
|          Tucson|         7880|
|    Indianapolis|         6915|
|       Nashville|         6178|
|     New Orleans|         5867|
|            Reno|         5097|
|        Edmonton|         4675|
|     Saint Louis|         4369|
|   Santa Barbara|         2989|
|           Boise|         2471|
|      Clearwater|         1900|
|        Metairie|         1539|
|Saint Petersburg|         1448|
|          Sparks|         1345|
|      Wilmington|         1263|
|        Franklin|         1227|
|       St. Louis|         1088|
|  St. Petersburg|         1052|
|         Brandon|          930|
+----------------+-------------+
only showing top 20 rows



In [4]:

checkin_df = spark.read.json('D:/china/intershipBigData/yelp/yelp_academic_dataset_checkin.json')
business_df = spark.read.json('D:/china/intershipBigData/yelp/yelp_academic_dataset_business.json')
joined_df = checkin_df.join(business_df, checkin_df.business_id == business_df.business_id)
merchant_checkin_frequency = joined_df.groupBy("name").agg(count("*").alias("checkin_count")).orderBy(col("checkin_count").desc())
top_merchants = merchant_checkin_frequency.limit(20)
top_merchants.show()





+--------------------+-------------+
|                name|checkin_count|
+--------------------+-------------+
|           Starbucks|          724|
|          McDonald's|          701|
|             Dunkin'|          505|
|              Subway|          453|
|           Taco Bell|          360|
|        CVS Pharmacy|          344|
|           Walgreens|          341|
|         Burger King|          334|
|             Wendy's|          331|
|                Wawa|          307|
|      Domino's Pizza|          286|
|       The UPS Store|          276|
|           Pizza Hut|          256|
|Enterprise Rent-A...|          223|
|   Papa John's Pizza|          192|
|         Great Clips|          179|
|      US Post Office|          174|
|        Jimmy John's|          174|
|                 KFC|          170|
|         Chick-fil-A|          162|
+--------------------+-------------+



In [ ]:

checkin_df = spark.read.json('D:/china/intershipBigData/yelp/yelp_academic_dataset_checkin.json')
checkin_df = checkin_df.withColumn("checkin_dates", to_date("date", "yyyy-MM-dd HH:mm:ss"))
yearly_checkin_freq = checkin_df.withColumn("Year", year("checkin_dates")).groupBy("Year").count().orderBy("Year")
yearly_checkin_freq.show()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `checkin_dates` cannot be resolved. Did you mean one of the following? [`date`, `business_id`].;
'Project [business_id#398, date#399, to_date('checkin_dates, Some(yyyy-MM-dd HH:mm:ss), Some(Africa/Casablanca), false) AS checkin_dates#402]
+- Relation [business_id#398,date#399] json


In [12]:

checkin_df = spark.read.json('D:/china/intershipBigData/yelp/yelp_academic_dataset_checkin.json')
checkin_df = checkin_df.withColumn("checkin_timestamp", to_timestamp("date", "yyyy-MM-dd HH:mm:ss"))
checkin_df = checkin_df.withColumn("HourOfDay", hour("checkin_timestamp"))
hourly_checkin_freq = checkin_df.groupBy("HourOfDay").agg(count("*").alias("CheckinFrequency")).orderBy("HourOfDay")
hourly_checkin_freq.show(5)


Py4JJavaError: An error occurred while calling o245.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 50) (neo-PC executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020-07-24 22:42:27, 2020-10-24 21:36:13, 2020-12-09 21:23:33, 2021-01-20 17:34:57, 2021-04-30 21:02:03, 2021-05-25 21:16:54, 2021-08-06 21:08:08, 2021-10-02 15:15:42, 2021-11-11 16:23:50' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.